In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import shutil
import natsort  #排序
import cv2
import torch
import random
import numpy as np
import pandas as pd
from albumentations import CenterCrop
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import torchvision
from torchvision import transforms
import torch.nn as nn

In [ ]:
###########################################
#图像文件分类,白光,荧光照片分为两类
##########################################


path = r'./Rock/'
fluorescence = r'./input/fluorescence'
white_light = r'./input/white_light'

files_list = os.listdir(path)

for file in files_list:
    filename, name_suffix = os.path.splitext(file)  # filename是文件名 suffix是文件后缀
    file_bmp_path = path + "/" + filename + '.bmp'
    file_jpg_path = path + "/" + filename + '.jpg'
    label1 = filename.split('-')[1]  # '-'后面的文件名
    label2 = label1.split('.')[0]  # '.'前面的文件名
    
    if label2 == '1':
        if name_suffix =='.bmp':
            shutil.copy(file_bmp_path, white_light)
        else:
            shutil.copy(file_jpg_path, white_light)
    else:
        if name_suffix =='.bmp':
            shutil.copy(file_bmp_path, fluorescence)
        else:
            shutil.copy(file_jpg_path, fluorescence)


In [ ]:
###################################################
#图像切割
###################################################

train_path = './input/white_light/'
zhongzhuan_path = './input/zhongzhuan/'
files_list = os.listdir(train_path)
files_list = natsort.natsorted(files_list)#排序
save_path = './input/imges_output/'

def imread(image):
    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #实现色彩空间转换
    image = image.astype(np.uint8)
    return np.array(image)

In [ ]:

for i in files_list:
    m = 0
    filename, name_suffix = os.path.splitext(i)  # filename是文件名 suffix是文件后缀
    main_name = filename.split('-')[0]  # '-'前面的主文件名
    main_name = int(main_name)
    
    if main_name < 321:
        main_name = str(main_name)
        img = Image.open(train_path + i)
        size = img.size
        n_equal_parts=3
        # 准备将图片切割成9张小图片
        weight = int(size[0] // n_equal_parts)
        height = int(size[1] // n_equal_parts)
        for j in range(n_equal_parts):
            for k in range(n_equal_parts):
                m+=1
                box = (weight * k, height * j, weight * (k + 1), height * (j + 1))
                region = img.crop(box)
                region.save(save_path + main_name +'_{}.bmp'.format(m))
    main_name = int(main_name)            
    if main_name > 321:
        main_name = str(main_name)
        img = imread(train_path + i)
        img = CenterCrop(1024, 1024, always_apply=False, p=1.0)(image=img)['image']#裁剪输入的中心部分
        cv2.imwrite(zhongzhuan_path + main_name + '-{}.jpg'.format(1), img)
        img = Image.open(zhongzhuan_path + i)
        size = img.size
        n_equal_parts=2# 准备将图片切割成4张小图片
        weight = int(size[0] // n_equal_parts)
        height = int(size[1] // n_equal_parts)
        for j in range(n_equal_parts):
            for k in range(n_equal_parts):
                m+=1
                box = (weight * k, height * j, weight * (k + 1), height * (j + 1))
                region = img.crop(box)
                region.save(save_path + main_name+'_{}.bmp'.format(m))
        
        
                
                
        
               

In [ ]:
######################################################
#图像池化压缩图像
#####################################################
data_path = './input/images_output/'
save_path = './input/images_output_cnn/'
files_list = os.listdir(data_path)
files_list = natsort.natsorted(files_list)

In [ ]:
def transform_invert(img_, transform_train):
    """
    将data 进行反transfrom操作
    :param img_: tensor
    :param transform_train: torchvision.transforms
    :return: PIL image
    """
    if 'Normalize' in str(transform_train):
        norm_transform = list(filter(lambda x: isinstance(x, transforms.Normalize), transform_train.transforms))
        mean = torch.tensor(norm_transform[0].mean, dtype=img_.dtype, device=img_.device)
        std = torch.tensor(norm_transform[0].std, dtype=img_.dtype, device=img_.device)
        img_.mul_(std[:, None, None]).add_(mean[:, None, None])

    img_ = img_.transpose(0, 2).transpose(0, 1)  # C*H*W --> H*W*C
    img_ = np.array(img_) * 255

    if img_.shape[2] == 3:
        img_ = Image.fromarray(img_.astype('uint8')).convert('RGB')
    elif img_.shape[2] == 1:
        img_ = Image.fromarray(img_.astype('uint8').squeeze())
    else:
        raise Exception("Invalid img shape, expected 1 or 3 in axis 2, but got {}!".format(img_.shape[2]) )

    return img_

#最大池化
def maxpoll(img_tensor):
    maxpool_layer = nn.MaxPool2d((2,2),stride=(2,2)) # input:(i, o, size) weights:(o, i , h, w)
    img_maxpool = maxpool_layer(img_tensor)
    return img_maxpool


In [ ]:
for i in files_list:
    m = 0
    filename, name_suffix = os.path.splitext(i)  # filename是文件名 suffix是文件后缀
    main_name = filename.split('_')[0]  # '-'前面的主文件名
    main_name =int(main_name)
    if main_name <321:
        a=1
        img = Image.open(data_path+i).convert('RGB')
        # 转换为向量
        img_transform = transforms.Compose([transforms.ToTensor()])
        img_tensor = img_transform(img)
        img_tensor.unsqueeze_(dim=0) # C*H*W to B*C*H*W
        img_pool = maxpoll(img_tensor)#最大池化
        img_pool = transform_invert(img_pool[0,0:3,...],img_transform)#向量还原为图片
        img_pool.save(save_path+i)
    else:
        shutil.copy(data_path + i, save_path)
        

        

In [ ]:
#########################################
#标签编码及制作新的csv文件
#########################################

In [ ]:
#数据label加载
df=pd.DataFrame(pd.read_csv('./rock_label.csv',encoding='GBK'))
df['样本类别'].value_counts()
#标签编码
le=LabelEncoder()
le.fit(df['样本类别'].values.tolist())
le.classes_
df['样本类别']=le.transform(df['样本类别'].values.tolist())
df['样本类别'].value_counts()

In [ ]:
le.inverse_transform([0, 1, 2, 3, 4, 5, 6])#查看编码对应的标签

In [ ]:
len_col= df.shape[0]+1
loc_index = 0
new_df = pd.DataFrame()

for loc in range(len_col):
    loc_index += 1
    if loc <289:
        for i in range(9):
            new_df = new_df.append(df[loc_index-1:loc_index])
            
    else:
        for i in range(4):
            new_df = new_df.append(df[loc_index-1:loc_index])
        

In [ ]:
path = r'./input/images_output_cnn/'
files_list = os.listdir(path)
files_list = natsort.natsorted(files_list)#排序
new_df['样本编号'] = files_list
new_df.columns = ['image_id', 'label']

In [ ]:
new_df.index=np.arange(0,new_df.shape[0],1)
new_df.index#重建索引

In [ ]:
#####################################
#增加flod列,用于5折交叉验证
#####################################
fold=pd.DataFrame(np.random.randint(0,5,new_df['image_id'].values.shape))
new_df['fold']=fold
new_df['fold'].value_counts()
new_df

In [ ]:
new_df.to_csv("./train.csv", encoding="GBK",index=False)

In [ ]:
new_df['fold'].value_counts()

In [ ]:
new_df['label'].value_counts()